In [74]:
import warnings
warnings.filterwarnings('ignore')

In [75]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

train=pd.read_csv("/opt/ml/input/data/train/train_ratings.csv")

csv_list = !ls *.csv
csv_list

['EASE@500.csv',
 'FFM@500.csv',
 'GRU4RecF@500.csv',
 'ItemKNN@500.csv',
 'NeuMF@500.csv',
 'Pop@500.csv',
 'SLIMElastic@500.csv']

In [76]:
df=pd.DataFrame()

for csv in csv_list:
    file = pd.read_csv(csv)
    df = pd.concat([df,file])

df.shape

(109487086, 2)

In [77]:
df.drop_duplicates(subset=['user','item'],inplace=True)
df.shape

(34713290, 2)

In [78]:
df = pd.concat([df,train[['user','item']]])
df.shape

(39867761, 2)

In [79]:
# Get a list of all unique users and items
users = df['user'].unique()
items = df['item'].unique()

In [54]:
user2idx = {k:v for v,k in enumerate(sorted(set(train.user)))}

uidx2id = {v:k for v,k in enumerate(sorted(set(train.user)))}

item2idx = {k:v for v,k in enumerate(sorted(set(train.item)))}

itemidx2id = {v:k for v,k in enumerate(sorted(set(train.item)))}

In [32]:
items

array([ 4370,  4886,    47, ...,  6430,  5423, 43934])

In [7]:
# Create a new dataframe to hold the negative samples
negative_samples = pd.DataFrame(columns=['user', 'item'])

In [8]:
negative_dict = dict()

In [11]:
len(users),len(items)

(31360, 6807)

In [91]:
user_items = df[df['user'] == 11]['item']

In [97]:
2428 in user_items.to_list()

True

In [98]:
2428 in [item for item in items if item not in user_items.to_list()]

False

In [12]:
# Iterate through the list of users
for user in tqdm(users):
    # Get all the items that the user has interacted with
    user_items = df[df['user'] == user]['item']
    user_items_num = train[train.user==user].shape[0]
    random.shuffle(items)
    negative_items = [item for item in items if item not in user_items][:user_items_num]
    negative_dict[user]=negative_items

100%|██████████████████████████████████████████████████████| 31360/31360 [1:01:46<00:00,  8.46it/s]


In [88]:
2428 in negative_dict[11]

True

In [13]:
len(negative_dict)

31360

In [34]:
result = pd.DataFrame(columns=['user','item'])

In [35]:
for u,i_li in tqdm(negative_dict.items()):
    result.loc[len(result),:]=(u," ".join(map(str,i_li)))

100%|███████████████████████████████████████████████████████| 31360/31360 [00:47<00:00, 658.41it/s]


In [36]:
result

,user,item
0,11,97328 3480 3718 5389 1185 45517 7564 7234 9356...
1,14,3734 6382 4368 2131 89780 3182 5398 55820 4931...
2,18,479 1831 2790 25929 53550 4040 65585 3668 6967...
3,25,78729 32022 46974 85354 4415 849 1227 94953 55...
4,31,1280 52967 4356 41014 26578 1961 56012 87192 2...
...,...,...
31355,138473,2580 5417 27032 849 89 3747 955 946 7218 1441 ...
31356,138475,27831 1379 61071 3521 3196 2539 45440 6619 32 ...
31357,138486,1721 7265 1588 55805 12 7482 4148 27005 2353 4...
31358,138492,1231 84944 2090 55814 33836 26578 880 80939 51...


In [37]:
b_flat = pd.DataFrame([[i, x] 
               for i, y in result['item'].apply(lambda x: x.split()).iteritems() 
                    for x in y], columns=list('IB'))
b_flat = b_flat.set_index('I')

result2 = result[['user']].merge(b_flat, left_index=True, right_index=True)

In [39]:
result2.columns=['user','item']

In [42]:
train2 = train[['user','item']]

In [43]:
train2['label']=1

In [44]:
result2['label']=0

In [85]:
df[(df.user==11) & (df.item==2428)]

,user,item
8,11,2428


In [63]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5154471 entries, 0 to 5154470
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   user    int64
 1   item    int64
 2   time    int64
dtypes: int64(3)
memory usage: 118.0 MB


In [67]:
result2.user = result2.user.astype(int)
result2.item = result2.item.astype(int)

In [70]:
train[(train.user==11) & (train.item==2428)]

,user,item,time
8,11,2428,1230782694


In [72]:
result2[(result2.user==11) & (result2.item==2428)]

,user,item,label
0,11,2428,0


In [68]:
train[['user','item']].merge(result2[['user','item']], on=['user','item'])

,user,item
0,11,2428
1,11,33585
2,11,2124
3,11,7173
4,11,1321
...,...,...
227160,138493,1203
227161,138493,3034
227162,138493,4874
227163,138493,224


In [58]:
result2[['user','item']].to_pickle("Neg_Sample.pkl")

In [46]:
all_df = pd.concat([train2,result2])

In [47]:
all_df.shape

(10308942, 3)

In [48]:
all_df.to_pickle('All_Pos_Neg_Interaction.pkl')

In [50]:
tt = pd.read_pickle('All_Pos_Neg_Interaction.pkl')

In [56]:
scp -P 2240 All_Pos_Neg_Interaction.pkl root@27.96.131.55:~

scp -P 2226 All_Pos_Neg_Interaction.pkl root@101.101.216.53:~

-recsys8: No such file or directory
